# Algoritmo di 2-approssimazione per TSP

In [92]:
!pip install ipython-autotime
#%load_ext autotime

You should consider upgrading via the 'c:\users\cip\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


In [93]:
import time
import copy
import math
from collections import defaultdict

Imposto la directory in cui sono presenti i dataset:

In [94]:
ds_dir = "tsp_dataset/"

Definisco una lista con i nomi dei file del dataset dato e le soluzioni ottime date per ciascun file, da confrontare poi con le soluzioni calcolate:

In [95]:
data = [
    ["burma14.tsp", 3323],
    ["ulysses16.tsp", 6859],
    ["ulysses22.tsp", 7013],
    ["eil51.tsp", 426],
    ["berlin52.tsp", 7542],
    ["kroD100.tsp", 21294],
    ["kroA100.tsp", 21282],
    ["ch150.tsp", 6528],
    ["gr202.tsp", 40160],
    ["gr229.tsp", 134602],
    ["pcb442.tsp", 50778],
    ["d493.tsp", 35002],
    ["dsj1000.tsp", 18659688]
]

In [96]:
class Node:
    def __init__(self, tag: int):
        self.tag = tag
        self.key = None
        self.parent = None
        self.isPresent = True
        self.index = tag # Track the index of the node in the heap instead of using list.index() method which is O(n)
        self.adjacencyList = []

    # For test
    def print(self):
        print("tag =", self.tag, "adjList=", self.adjacencyList, "key=", self.key)

class Graph:
    def __init__(self):
        self.nodes = defaultdict(Node)

    def createNodes(self, nums: int):
        for i in range(0, nums): # nums+1 in order to cover the last node
            self.nodes[i] = Node(i)

    def addNode(self, tag:int, adjTag:int, adjCost):
        #self.nodes[tag].adjacencyList.append([self.nodes[adjTag], adjCost])
        self.nodes[adjTag].adjacencyList.append([self.nodes[tag], adjCost]) # Graph is undirected

    def buildGraphTSP(self, input_file):
        t, V = parser(input_file)
        self.createNodes(len(V))
        for v in V:
            tag_v = int(v[0])
            U = copy.deepcopy(V)
            U.remove(v)
            for u in U:
                tag_u = int(u[0])
                curr_weight = weight(v[1], u[1], t)
                self.addNode(tag_v, tag_u, curr_weight)

        return t, V

In [97]:
# ArrayHeap object extends list
class ArrayHeap(list):
    def __init__(self, array):
        super().__init__(array)
        self.heapSize = len(array)


class MinHeap:
    def __init__(self, array: list, root: Node):
        self.arrayHeap = ArrayHeap(array)
        # Check if the root node is not the first
        if self.arrayHeap[0] != self.arrayHeap[root.tag]:  # reset the starting node and update all indexes
            rootNode = self.arrayHeap[root.tag]
            self.arrayHeap.remove(rootNode)
            self.arrayHeap.insert(0, rootNode)
            for i in range(0, self.arrayHeap.heapSize):
                self.arrayHeap[i].index = i

    # All the following methods work with zero based array. Hence, we need to handle separately odd and even indexes.
    def parent(self, i: int):
        if i % 2 == 0:  # even
            return i // 2 - 1
        else:
            return i // 2

    def left(self, i):
        return 2 * i + 1

    def right(self, i):
        return 2 * i + 2

    # Execution time: O(lg n)
    def minHeapify(self, i: int):
        l = self.left(i)
        r = self.right(i)
        if l <= self.arrayHeap.heapSize - 1 and self.arrayHeap[l].key < self.arrayHeap[i].key:
            minimo = l
        else:
            minimo = i
        if r <= self.arrayHeap.heapSize - 1 and self.arrayHeap[r].key < self.arrayHeap[minimo].key:
            minimo = r
        if minimo != i:
            self.arrayHeap[i].index, self.arrayHeap[minimo].index = minimo, i  # Update indexes
            self.arrayHeap[i], self.arrayHeap[minimo] = self.arrayHeap[minimo], self.arrayHeap[i]
            self.minHeapify(minimo)

    def bubbleUp(self, index: int):
        parent = self.parent(index)
        current = index
        while current > 0 and self.arrayHeap[parent].key > self.arrayHeap[current].key:
            self.arrayHeap[current].index, self.arrayHeap[parent].index = parent, current  # Update indexes
            self.arrayHeap[current], self.arrayHeap[parent] = self.arrayHeap[parent], self.arrayHeap[current]
            current = parent
            parent = self.parent(parent)

    # Execution time: O(lg n)
    # First we update the heap structure, then we remove the last element.
    def extractMin(self):
        if self.arrayHeap.heapSize < 1:
            print("Error: extractMin underflow")
            return
        else:
            minimum = self.arrayHeap[0]  # Save the minimum node
            self.arrayHeap[0].isPresent = False  # Set its flag to false
            # Swap the first node and right most one
            self.arrayHeap[0], self.arrayHeap[self.arrayHeap.heapSize - 1] = self.arrayHeap[
                                                                                 self.arrayHeap.heapSize - 1], \
                                                                             self.arrayHeap[0]
            self.arrayHeap[0].index = 0  # Update its index
            self.arrayHeap.heapSize -= 1  # Decreasing heapsize
            self.minHeapify(0)  # Call minHeapify in order to move the new first node to the correct position

            return minimum

In [98]:
def parser(file):

  lines = open(ds_dir + file, "r").readlines()
  index_start_coordinates = 0
  cont = 0
  V = []

  for line in lines:
      cont += 1
      if line.startswith("EOF") or line.startswith(" EOF"):
          break
      elif line.startswith("DIMENSION"):
          n = int(line.split(":")[1][1:])
      elif line.startswith("EDGE_WEIGHT_TYPE"):
          t = line.split(":")[1][1:-1]  # TODO: remove space at the end
      elif line.startswith("NODE_COORD_SECTION"):
          index_start_coordinates = cont
      elif index_start_coordinates > 0:
          V.append((int(line.split()[0]) - 1,
                    [float(line.split()[1]), float(line.split()[2])]))  # (i, [x_value, y_value])
  # n = int(lines[3].split()[1]) #.split()[0] # extract number of vertexes
  # t = lines[4].split()[1]

  return t, V

In [99]:
def weight (u, v, t):
  if t == 'EUC_2D':
    return round(math.sqrt(sum([(a - b) ** 2 for a, b in zip(u, v)])))
  else:
    PI = 3.141592
    deg_xu = int(u[0])
    min_xu = u[0] - deg_xu
    rad_xu = PI * (deg_xu + 5.0 * min_xu/ 3.0) / 180.0

    deg_yu = int(u[1])
    min_yu = u[1] - deg_yu
    rad_yu = PI * (deg_yu + 5.0 * min_yu/ 3.0) / 180.0

    deg_xv = int(v[0])
    min_xv = v[0] - deg_xv
    rad_xv= PI * (deg_xv + 5.0 * min_xv/ 3.0) / 180.0

    deg_yv = int(v[1])
    min_yv = v[1]- deg_yv
    rad_yv = PI * (deg_yv + 5.0 * min_yv/ 3.0) / 180.0

    RRR = 6378.388
    q1 = math.cos(rad_yu - rad_yv)
    q2 = math.cos(rad_xu - rad_xv)
    q3 = math.cos(rad_xu + rad_xv)
    return (int) (RRR * math.acos(0.5 * ((1.0 + q1) * q2 - (1.0 - q1) * q3)) + 1.0)

In [100]:
def ham_circ_weight(V, ham_circ):

    total_weight = 0
    for j in range(1, len(ham_circ)):
        i = j - 1
        total_weight += weight(V[ham_circ[i]][1], V[ham_circ[j]][1] , t) #get_weight(ham_circ[i], ham_circ[j])

    return total_weight

In [101]:
def MSTPrim(g: Graph, r: Node):

    for node in g.nodes.values():
        node.key = math.inf # Set key. Parent is already set through Node constructor.
        node.parent = r

    r.key = 0

    q = MinHeap(list(g.nodes.values()), r) # Pass also the root node in order to build the heap starting from it

    while q.arrayHeap.heapSize != 0:
        u = q.extractMin()
        for v in u.adjacencyList:
            if v[0].isPresent and v[1] < v[0].key:
                v[0].parent = u
                v[0].key = v[1]
                q.bubbleUp(v[0].index) # bubbleUp maintains the minheap condition

    return g

Definisco la funzione MST_TO_TREE che, data in input la mappa dei predecessori ciascun nodo, la converte in un albero

In [102]:
def MST_TO_TREE(predec):

    if len(predec) == 0:
        return None

    if len(predec) == 1:
        nodo, padre = predec.popitem()
        return {padre: [nodo]}

    nodo, padre = predec.popitem()
    tree = MST_TO_TREE(predec)

    if padre in tree:
        tree[padre].append(nodo)
    else:
        tree[padre] = [nodo]

    return tree

Implemento la funzione di 2-approssimazione che, dato in input il grafo completo pesato G, risolve il problema TSP sul grafo G usando il suo albero di copertura minimo di G.
Ritorna una soluzione 2-approssimata, ovvero il valore massimo di questa è 2 volte la soluzione ottima, definita precedentemente per ciascun file nella lista `data`.
Questo metodo restituisce un ciclo hamiltoniano che visita tutti i nodi.

Definisco la funzione preorder che, presi in input tree (la mappa dei successori) e u (vertice di partenza), ritorna una lista della visita in profondità dell'albero a partire dal nodo passato.

In [103]:
def preorder(tree, u):

    if u not in tree:
        return [u]

    A = [u]
    for v in tree[u] :
        if v != u :
            A = A + preorder(tree, v)

    return A


Eseguo l'algortimo appena definito per ciascun file presente nel dataset:

In [104]:
for file, optimal_solution in data:

    start_time = time.time()

    ###################

    result = Graph()
    t, V = result.buildGraphTSP(file)

    current_solution = MSTPrim(result, result.nodes.get(0))

    parents = dict()
    for i in current_solution.nodes.values():
        parents[i.tag] = i.parent.tag

    TREE = MST_TO_TREE(parents)
    HAM_CYCLE = preorder(TREE, 0)
    HAM_CYCLE.append(0)

    current_solution = ham_circ_weight(V, HAM_CYCLE)

    ###################

    total_time = time.time() - start_time
    total_time = '%.2E' % total_time

    print("File name: ", file)
    print("Optimal solution: ", optimal_solution)
    print("Costo soluzione: ", str(current_solution))
    print("Tempo di esecuzione: ", str(total_time))
    #print("Percentuale di errore: ", str(errore))

    print()

File name:  burma14.tsp
Optimal solution:  3323
Costo soluzione:  4003
Tempo di esecuzione:  5.01E-03

File name:  ulysses16.tsp
Optimal solution:  6859
Costo soluzione:  7788
Tempo di esecuzione:  5.00E-03

File name:  ulysses22.tsp
Optimal solution:  7013
Costo soluzione:  8308
Tempo di esecuzione:  1.00E-02

File name:  eil51.tsp
Optimal solution:  426
Costo soluzione:  605
Tempo di esecuzione:  4.50E-02

File name:  berlin52.tsp
Optimal solution:  7542
Costo soluzione:  10402
Tempo di esecuzione:  4.20E-02

File name:  kroD100.tsp
Optimal solution:  21294
Costo soluzione:  28599
Tempo di esecuzione:  1.53E-01

File name:  kroA100.tsp
Optimal solution:  21282
Costo soluzione:  30516
Tempo di esecuzione:  1.15E+00

File name:  ch150.tsp
Optimal solution:  6528
Costo soluzione:  9126
Tempo di esecuzione:  3.44E-01

File name:  gr202.tsp
Optimal solution:  40160
Costo soluzione:  52615
Tempo di esecuzione:  6.75E-01

File name:  gr229.tsp
Optimal solution:  134602
Costo soluzione:  179